In [1]:
import psycopg2
import numpy as np
import pandas as pd
import datetime as dt
import cx_Oracle
import csv

In [2]:
con =  cx_Oracle.connect('comprasweb_ij/Xxxx1234!@10.10.2.223/MKTG')
print(con.version)

df = pd.read_sql_query('''
SELECT * FROM 
(
---- Query Importe anual
SELECT 
	--(DENSE_RANK() OVER(ORDER BY IMPORTE_TOTAL DESC)) AS ranking_cliente,
	de.EM_ID_CLIENTE,
	de.EM_ID,
	de.EM_NOMBRE,
	s.importe_total AS importe_365
FROM
	EMPLEO_ES.EMPRESAS_MATRICES de 
INNER JOIN
	(--s
		SELECT	
			COD_EMPRESA,
			sum(s.importe) AS importe_total
		FROM	
			DWHPRO.FACT_ENTERPRISE_SALES s
		WHERE	
			to_date(s.FECHA_IMPUTACION, 'yyyymmdd') BETWEEN CURRENT_DATE -366 AND CURRENT_DATE -1
			having sum(s.importe) > 0
		GROUP BY
			s.COD_EMPRESA 
	)s
	ON de.EM_ID_CLIENTE = s.COD_EMPRESA 
WHERE  
de.EM_PRUEBA <> 1 
AND de.EM_ESTADO = 1
)
--WHERE 
--ranking_cliente <= 100
--ORDER BY 
--RANKING_CLIENTE ASC 
''', con)
importe_df = df
df_save = df
importe_df.head()

con.close()

11.2.0.3.0


In [3]:
importe_df.head()

,RANKING_CLIENTE,EM_ID_CLIENTE,EM_ID,EM_NOMBRE,IMPORTE_365
0,1,8600653408,48660368,Eurovendex,625071.82
1,1,8600653408,9574632660,Adecco No Redireccionamiento,625071.82
2,1,8600653408,27958188850,Spring,625071.82
3,1,8600653408,79620258,Adecco,625071.82
4,2,8600632686,48723483,EUROFIRMS,342165.00


In [4]:
importe_df.shape

(195, 5)

In [5]:
con =  cx_Oracle.connect('comprasweb_ij/Xxxx1234!@10.10.2.223/MKTG')
print(con.version)

df = pd.read_sql_query('''
--Cantidad de experiencias con empresas normalizadas y review 
--a 
SELECT 
	t.ID_EXPERIENCIA_NORM,
	t.DESC_EXPERIENCIA_NORMALIZADO,
	MEDIAN(t.RATINGEVENTOBJECTSCORE) mediana,
	sum(t.RATINGEVENTOBJECTSCORE)/
	count(t.RATINGEVENTOBJECTSCORE) promedio,
	count(t.RATINGEVENTOBJECTSCORE) cantidad
FROM 
(--t --SI NO FUNCIONA AGREGAR DESDE ACA A JUPYTER Y SACAR PROMEDIOS DESDE EL NOTEBOOK
SELECT * 
FROM --DISTINCT para empresas
(
SELECT 
	RATINGEVENTOBJECTTARGETID,
	s.DT,
	TRUNC(s.RATINGEVENTOBJECTINSTANT) AS fecha_review,
	s.RATINGEVENTOBJECTSCORE,
	s.REVIEWNAME,
	s.REVIEWVALUE
FROM DWHPRO.COMPANY_REVIEW s 
--WHERE to_date(DT) BETWEEN to_date('01/09/2018') AND to_date('31/12/2019')
)t 
pivot (
min(REVIEWVALUE) value FOR REVIEWNAME IN ('title', 'description')
)
INNER JOIN 
DWHPRO.EXPERIENCIAS_COMPANY_NORM e ON ID_EXPERIENCIA = RATINGEVENTOBJECTTARGETID
WHERE 1=1
--AND CREATED_AT >= TO_DATE('2019-01-01','YYYY-mm-dd')
AND NORMALIZATION_MODEL IS NOT NULL
AND SCORE >= 0.7
AND ID_EXPERIENCIA_NORM <> '-1'
)t 
GROUP BY 
t.ID_EXPERIENCIA_NORM,
t.DESC_EXPERIENCIA_NORMALIZADO
''', con)
score_df = df
df_save_2 = df
score_df.head()

con.close()

11.2.0.3.0


In [6]:
score_df[score_df['DESC_EXPERIENCIA_NORMALIZADO']=='Mercadona']

,ID_EXPERIENCIA_NORM,DESC_EXPERIENCIA_NORMALIZADO,MEDIANA,PROMEDIO,CANTIDAD
414,309,Mercadona,5.0,4.235902,4256


In [8]:
con =  cx_Oracle.connect('comprasweb_ij/Xxxx1234!@10.10.2.223/MKTG')
print(con.version)

df = pd.read_sql_query('''
--perfiles, matrices y normalizacion
SELECT 
ID_PUESTO_NORM ID_EXPERIENCIA_NORM,
DESC_OFERTA_NORMALIZADO DESC_EXPERIENCIA_NORMALIZADO,
EP_MATRIZ EM_ID
FROM DWHPRO.OFERTAS_COMPANY_NORM cn 
LEFT JOIN 
EMPLEO_ES.EMPRESAS_PERFILES ep ON cn.ID_OFERTA = ep.EP_ID
WHERE 1=1
AND NORMALIZATION_MODEL IS NOT NULL
AND SCORE >= 0.7
GROUP BY 
ID_PUESTO_NORM,
DESC_OFERTA_NORMALIZADO,
EP_MATRIZ
''', con)
perfil_df = df
df_save_3 = df
perfil_df.head()

con.close()

11.2.0.3.0


In [9]:
score_matriz = perfil_df.merge(score_df,how='inner',on = 'DESC_EXPERIENCIA_NORMALIZADO') 

In [10]:
score_matriz.shape

(29651, 7)

In [11]:
score_matriz.head()

,ID_EXPERIENCIA_NORM_x,DESC_EXPERIENCIA_NORMALIZADO,EM_ID,ID_EXPERIENCIA_NORM_y,MEDIANA,PROMEDIO,CANTIDAD
0,2498,Johnson Controls,4.879788e+07,2498,5.0,4.328,125
1,2498,Johnson Controls,3.070372e+10,2498,5.0,4.328,125
2,2498,Johnson Controls,2.519039e+09,2498,5.0,4.328,125
3,2498,Johnson Controls,2.565047e+09,2498,5.0,4.328,125
4,2498,Johnson Controls,4.838940e+07,2498,5.0,4.328,125


In [12]:
big_frame = score_matriz.merge(importe_df,how='inner',on = 'EM_ID')

In [13]:
big_frame.shape

(362, 11)

### Exportar data

In [224]:
big_frame.to_csv('score_top_ingresos_cruce_em_id_norm.csv', index= False)
#, sep=';')

In [34]:
score_matriz.to_csv('score_top_reviews_em.csv', index= False)
#, sep=';')

In [35]:
score_df.to_csv('score_top_reviews.csv', index= False)
#, sep=';')

In [37]:
importe_df.to_csv('score_top_ingresos.csv', index= False)
#, sep=';')

In [50]:
#Load the data 
df_csv = pd.read_csv('exp_norm_2034.csv')#, encoding='latin-1', sep='|', header=None)

In [52]:
#Load the data 
df_csv2 = pd.read_csv('reviews_all_data.csv')#, encoding='latin-1', sep='|', header=None)

In [66]:
#Load the data 
df_csv4 = pd.read_csv('ds_2034.csv')#, encoding='latin-1', sep='|', header=None)

In [77]:
df_csv = df_csv.rename(columns={"ne_exp_id": "EXPERIENCE_ID"})